In [1]:
from scipy.stats import linregress
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
import pandas as pd
import pymannkendall as mk
import folium
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
from matplotlib_scalebar.scalebar import ScaleBar

In [3]:
df = pd.read_csv('GWS.csv')

In [4]:
def trend_test(row):
    lat, lon = row['lat'], row['lon']
    time_series = row.iloc[2:]
    
    # Perform the Seasonal Mann-Kendall test on the time series
    result = mk.seasonal_test(time_series, period=12)
    
    # Calculate the mean annual trend using Sen's slope
    annual_trend = mk.sens_slope(time_series.iloc[::12])  # Select only annual data points
    
    # Extract the p-value, Kendall's tau, and mean annual trend from the test result
    p_value = result.p
    tau = result.Tau
    sen_slope = annual_trend.slope
    return pd.Series([lat, lon, p_value, tau, sen_slope])

# Apply the trend_test function on each row of the DataFrame
result_df = df.apply(trend_test, axis=1)

# Rename the columns of the result DataFrame
result_df.columns = ['lat', 'lon', 'p_value', 'tau', 'sen_slope']

print(result_df)

         lat     lon       p_value       tau  sen_slope
0     27.875  31.375  0.000000e+00  0.699495   4.873367
1     27.875  31.625  0.000000e+00 -0.676768  -3.395874
2     27.875  31.875  2.707465e-10 -0.404040  -1.510314
3     27.875  32.125  0.000000e+00  0.780303   6.792844
4     27.875  32.375  0.000000e+00  0.782828   7.306849
...      ...     ...           ...       ...        ...
6197  51.125  18.875  4.884152e-01 -0.045455   3.071008
6198  51.125  19.125  3.320643e-01 -0.063131   2.263581
6199  51.125  19.375  2.800145e-02 -0.141414   0.303642
6200  51.125  19.625  5.252571e-03 -0.179293  -0.718394
6201  51.125  19.875  3.397626e-04 -0.229798  -1.631012

[6202 rows x 5 columns]


In [8]:
result_df= result_df[result_df['sen_slope'] < -.]
result_df

,lat,lon,p_value,tau,sen_slope
